In [21]:
import numpy as np

In [22]:
grid_size = 4
min_delta = 0.001
actions = ['up','down','left','right']
policies = np.random.choice(actions, size=(grid_size,grid_size))
policies[0,0] = policies[grid_size-1, grid_size-1] = ''
v = np.zeros((grid_size, grid_size))
probs = [0.25]*4

In [23]:
def v_next(v, horizontal, vertical):
    if horizontal < 0:
        horizontal = 0
    elif horizontal >= grid_size:
        horizontal = grid_size - 1
    if vertical < 0:
        vertical = 0
    elif vertical >= grid_size:
        vertical = grid_size - 1
    return v[horizontal, vertical]

In [24]:
while True:
    delta = 0
    for i in range(grid_size):
        for j in range(grid_size):
            if not (i==j==0 or i==j==grid_size-1):
                new_v = v.copy()
                new_v[i,j] = np.dot(probs, [v_next(v,i,j+1), v_next(v,i,j-1), v_next(v,i-1,j), v_next(v,i+1,j)]) - 1
                delta = max(delta, abs(new_v[i,j]-v[i,j]))
                v = new_v
    if delta < min_delta:
        break

In [25]:
v

array([[  0.        , -13.99330608, -19.99037659, -21.98940765],
       [-13.99330608, -17.99178568, -19.99108113, -19.99118312],
       [-19.99037659, -19.99108113, -17.99247411, -13.99438108],
       [-21.98940765, -19.99118312, -13.99438108,   0.        ]])